In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

from tensorflow import keras
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import librosa
import IPython.display as ipd
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
import librosa
import os
from sklearn.model_selection import train_test_split
import numpy as np
# from tqdm import tqdm
from keras import optimizers

In [3]:
tf.config.experimental.get_memory_info('GPU:0')['current'] / 10**9

0.0

In [4]:
emotions_ravdess={
  '01': 1,
  '02': 2,
  '03': 3,
  '04': 4,
  '05': 5,
  '06': 6,
  '07': 7,
  '08': 8
}

emotions_to_string={
  1:'neutral',
  2:'calm',
  3:'happy',
  4:'sad',
  5:'angry',
  6:'fearful',
  7:'disgust',
  8:'surprised'
}

emotions_cremad={
  'ANG':5,
  'DIS':7,
  'NEU':1,
  'HAP':3,
  'FEA':6,
  'SAD':4
}

# observed_emotions=[1, 3, 4, 5, 6, 7]
observed_emotions=[1, 3, 5, 4]

In [5]:
MFCC_NUM = 100
MFCC_MAX_LEN = 500
SAMPLING_RATE = 44100
feature_dim_1 = MFCC_NUM
feature_dim_2 = MFCC_MAX_LEN
channel = 1

In [6]:
def extract_feature(wave, max_len=MFCC_MAX_LEN):
    mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [7]:
X = []
y = []

def append_X_Y(label, wave):
    y.append(label)
    mfcc = extract_feature(wave)
    X.append(mfcc)

In [8]:
for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\ravdess\\Actor_*\\*.wav"):
    file_name=os.path.basename(file)
    
    label=emotions_ravdess[file_name.split("-")[2]] 
#     label=str(emotions_ravdess[file_name.split("-")[2]]) + '_' + str(emotions_intensity_ravdess[file_name.split("-")[3]]) 
    if emotions_ravdess[file_name.split("-")[2]] not in observed_emotions:
        continue
    wave, sr = librosa.load(file, mono=True, sr=44100)
    wave = wave[::3]
    append_X_Y(label, wave)
    

for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\cremad\\*.wav"):
    file_name=os.path.basename(file)
    label=emotions_cremad[file_name.split("_")[2]] 
    if emotions_cremad[file_name.split("_")[2]] not in observed_emotions:
        continue
    wave, sr = librosa.load(file, mono=True, sr=44100)
    wave = wave[::3]
    append_X_Y(label, wave)

In [9]:
print(X[0].shape)
print("---------------------------------")
# print(X)
print("---------------------------------")
print(X[0])

(100, 500)
---------------------------------
---------------------------------
[[-917.5364 -917.5364 -917.5364 ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 ...
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]]


In [10]:
X = np.array(X)
y = np.array(y)

In [11]:
print(X[0].shape)
print("---------------------------------")
# print(X)
print("---------------------------------")
print(X[0])

(100, 500)
---------------------------------
---------------------------------
[[-917.5364 -917.5364 -917.5364 ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 ...
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=True, shuffle=True)

In [13]:
X_train.shape

(4032, 100, 500)

In [14]:
X_train = tf.expand_dims(X_train, axis=-1)

In [ ]:
X_test = tf.expand_dims(X_test, axis=-1)

In [15]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17235938373389096642,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2254123828
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4090237825086269367
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [16]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(120, (3, 3), activation='relu'))

In [17]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [18]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.25)

Epoch 1/10
95/95 [==============================] - 11s 58ms/step - loss: 1.5284 - accuracy: 0.4134 - val_loss: 3.0722 - val_accuracy: 0.2748
Epoch 2/10
95/95 [==============================] - 5s 51ms/step - loss: 1.0879 - accuracy: 0.5202 - val_loss: 1.0731 - val_accuracy: 0.4940
Epoch 3/10
95/95 [==============================] - 5s 51ms/step - loss: 0.9450 - accuracy: 0.5972 - val_loss: 0.9320 - val_accuracy: 0.5893
Epoch 4/10
95/95 [==============================] - 5s 51ms/step - loss: 0.8877 - accuracy: 0.6187 - val_loss: 0.9278 - val_accuracy: 0.6161
Epoch 5/10
95/95 [==============================] - 5s 51ms/step - loss: 0.8306 - accuracy: 0.6488 - val_loss: 0.9202 - val_accuracy: 0.6081
Epoch 6/10
95/95 [==============================] - 5s 51ms/step - loss: 0.8192 - accuracy: 0.6604 - val_loss: 0.8736 - val_accuracy: 0.6349
Epoch 7/10
95/95 [==============================] - 5s 51ms/step - loss: 0.7458 - accuracy: 0.6958 - val_loss: 1.0252 - val_accuracy: 0.5635
Epoch 8/10
9

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 498, 32)       320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 166, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 164, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 52, 120)        69240     
_________________________________________________________________
flatten (Flatten)            (None, 49920)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [ ]:
# MFCC_NUM = 13
# MFCC_MAX_LEN = 216
MFCC_NUM = 50
MFCC_MAX_LEN = 259
SAMPLING_RATE = 44100
feature_dim_1 = MFCC_NUM
feature_dim_2 = MFCC_MAX_LEN
channel = 1

In [ ]:
X = []
y = []

def append_X_Y(label, wave, sr):
    y.append(label)
    mfcc = extract_feature(wave, sr)
    X.append(mfcc)

In [ ]:
def extract_feature(wave, sr, max_len=MFCC_MAX_LEN):
    mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=sr)
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [ ]:
for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\ravdess\\Actor_*\\*.wav"):
    file_name=os.path.basename(file)
    label=emotions_ravdess[file_name.split("-")[2]] 
#     label=str(emotions_ravdess[file_name.split("-")[2]]) + '_' + str(emotions_intensity_ravdess[file_name.split("-")[3]]) 
    if emotions_ravdess[file_name.split("-")[2]] not in observed_emotions:
        continue
    wave, sr = librosa.load(file, sr=22050*2)
    wave = wave[::3]
    append_X_Y(label, wave, sr)
    


for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\cremad\\*.wav"):
    file_name=os.path.basename(file)
    label=emotions_cremad[file_name.split("_")[2]] 
    if emotions_cremad[file_name.split("_")[2]] not in observed_emotions:
        continue
    wave, sr = librosa.load(file, sr=22050*2)
    wave = wave[::3]
    append_X_Y(label, wave, sr)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=42, shuffle=True)

In [ ]:
X_train.shape

In [ ]:
# X_train = np.expand_dims(X_train,axis=3)
# X_test = np.expand_dims(X_test,axis=3)
X_train = np.expand_dims(X_train,axis=-1)
X_test = np.expand_dims(X_test,axis=-1)

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
#  Testing new model
model = Sequential()
model.add(layers.Conv2D(32, (3, 3),strides=(1,1),padding="same",
                 input_shape=(50, 259, 1)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3),strides=(1,1),padding="same",))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3),strides=(1,1),padding="same",))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(512, (3, 3),strides=(1,1),padding="same",))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(512, (2, 2),strides=(1,1),padding="same",))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(7))
model.add(layers.Activation('softmax'))

In [ ]:
#  Testing new model 2
model = Sequential()
model.add(layers.Conv2D(64, 5, strides=2,padding='same',
                 input_shape=(13, 216, 1)))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, 5,strides=2,padding='same',))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, 5,strides=2,padding='same',))
model.add(layers.Activation('relu'))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())

model.add(layers.Dense(7))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10, activation='softmax'))
model.add(layers.Activation('softmax'))

In [ ]:
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=16, epochs=50, validation_split=0.25)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

In [ ]:
import seaborn as sn
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, f1_score

In [ ]:

XTestProcessed = np.expand_dims(X_test, axis=-1)

In [ ]:
y_pred = model.predict(XTestProcessed)

In [ ]:
confusion_emotions = ['angry', 'happy', 'neutral', 'angry', 'angry', 'angry']

emotions_to_string={
  1:'neutral',
  3:'happy',
  5:'angry',
  6:'fearful',
  7:'disgust',
}

cm=confusion_matrix(y_test,np.argmax(y_pred,axis=-1))

print(emotions_to_string[np.argmax(y_pred[0],axis=0)])

df_cm=pd.DataFrame(cm,index=[i for i in emotions_to_string],columns=[i for i in emotions_to_string])
plt.figure(figsize=(10,7))
sn.heatmap(df_cm,annot=True)

In [ ]:
XTestProcessed = np.expand_dims(X_test, axis=-1)
y_pred = model.predict(XTestProcessed)

cnt_all = 0
cnt_true = 0

for i in range(len(y_pred)):
    classes_x=np.argmax(y_pred[i],axis=0)
    if (emotions_to_string[classes_x]==emotions_to_string[y[i]] and emotions_to_string[y[i]]=='angry'):
        cnt_true+=1
    if (emotions_to_string[classes_x]=='angry'):
        cnt_all+=1
#     print(emotions_to_string[classes_x], '     ',emotions_to_string[y[i]])

print(cnt_true/cnt_all)

In [ ]:
f1_score(y_test,np.argmax(y_pred,axis=-1),average='weighted')

In [ ]:
print(test_acc)

In [ ]:
import pickle

In [ ]:
filename = 'cnn_model_all_emotions_v2.pkl'
pickle.dump(model, open(filename, 'wb'))

In [20]:
model.save("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\cnn_model_v2\\")

INFO:tensorflow:Assets written to: C:\Users\User\Desktop\diploma\diploma\backend\emotion-recognition\cnn_model_v2\assets


In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0.7, 1)
plt.legend()

plt.show()

# test
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

In [ ]:
newFile = "C:/Users/User/Desktop/diploma/diploma/backend/speech-diarization/try/47_test.wav"
newFile = "C:/Users/User/Desktop/diploma/diploma/backend/emotion-recognition/ravdess/Actor_04/03-01-03-02-01-01-04.wav"

In [ ]:
wave, sr = librosa.load(newFile, mono=True, sr=44100)
wave = wave[::3]
mfcc = extract_feature(wave, sr)
X_test = mfcc.reshape(1, feature_dim_1, feature_dim_2, channel)
preds = model.predict(X_test)[0]
classes_x=np.argmax(preds,axis=0)
emotions_to_string[classes_x]

In [ ]:
Emotion_labels = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']
cnt_emotion_labels = [0, 0, 0, 0, 0, 0, 0, 0]

emotions_ravdess={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}


for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\ravdess\\Actor_*\\*.wav"):
    file_name=os.path.basename(file)
    cnt_emotion_labels[Emotion_labels.index(emotions_ravdess[file_name.split("-")[2]])]=cnt_emotion_labels[Emotion_labels.index(emotions_ravdess[file_name.split("-")[2]])]+1
  

plt.bar(Emotion_labels, cnt_emotion_labels) 
plt.show()

In [ ]:
Emotion_labels = ['ANG', 'DIS', 'NEU', 'HAP', 'FEA', 'SAD']
cnt_emotion_labels = [0, 0, 0, 0, 0, 0]


for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\cremad\\*.wav"):
    file_name=os.path.basename(file)
    cnt_emotion_labels[Emotion_labels.index(file_name.split("_")[2])]=cnt_emotion_labels[Emotion_labels.index(file_name.split("_")[2])]+1
    

plt.bar(Emotion_labels, cnt_emotion_labels) 
plt.show()

In [ ]:
# for testing new model 1

model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(50, 259, channel)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', strides=(1,1), padding=0))
model.add(layers.Conv2D(256, (3, 3), activation='relu', strides=(1,1), padding=0))
model.add(layers.Conv2D(512, (3, 3), activation='relu', strides=(1,1), padding=0))
model.add(layers.Conv2D(512, (2, 2), activation='relu', strides=(1,1), padding=0))
model.add(layers.MaxPool2D((2, 2)))

In [ ]:
# for testing new model 2

model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(14)) # Target class number
model.add(Activation('softmax'))
opt = tf.keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)